## Mask R-CNN - Train on NewShapes Dataset

### Notes from implementation
 


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import tensorflow as tf
import keras.backend as KB
import numpy as np
from mrcnn.datagen     import data_generator, load_image_gt
from mrcnn.callbacks   import get_layer_output_1,get_layer_output_2
from mrcnn.utils       import mask_string
import mrcnn.visualize as visualize
from mrcnn.prep_notebook import prep_oldshapes_train, load_model
# model_file = "E:\Models\mrcnn_oldshape_train_logs\TrainMRCNN\mask_rcnn_shapes_1418.h5"
# model_file = "E:\Models\mrcnn_oldshape_train_logs\shapes20180606T1150\mask_rcnn_shapes_0008.h5"
model, dataset_train, dataset_val, train_generator, val_generator, config = prep_oldshapes_train(init_with = 'last', FCN_layers = True, batch_sz = 16)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
 windows  Windows
Tensorflow Version: 1.4.0   Keras Version : 2.1.4 
 Initialize config object - super
(56, 56)
>>> Initialize model WITHOUT MASKING LAYERS!!!!
    set_log_dir: Checkpoint path set to : E:\models\mrcnn_oldshape_train_logs\shapes20180615T1939\mask_rcnn_shapes_{epoch:04d}.h5
    set_log_dir: self.epoch set to 0 

>>> Resnet Graph 
     Input_image shape : (?, 128, 128, 3)
     After ZeroPadding2D  : (?, 134, 134, 3) (?, 134, 134, 3)
     After Conv2D padding : (?, 64, 64, 64) (?, 64, 64, 64)
     After BatchNorm      : (?, 64, 64, 64) (?, 64, 64, 64)
     C1 Shape: (?, 32, 32, 64) (?, 32, 32, 64)
     C2 Shape:  (?, 32, 32, 256) (?, 32, 32, 256)
     C3 Shape:  (?, 16, 16, 512) (?, 16, 16, 512)
     C4 Shape:  (?, 8, 8, 1024) (?, 8, 8, 1024)
     C5 Shape:  (?, 4, 4, 2048) (?, 4, 4, 2048)

>>> Feature Pyramid Network (FPN) Graph 
     FPN P2 shape : (None, 32, 32, 256)
     FPN P3 shap

TypeError: reduce_max() got an unexpected keyword argument 'keepdims'

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import sys
import random
import math
import re
import  gc
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
# import tensorflow as tf
# import keras
import pprint
# import keras.backend as KB
sys.path.append('../')

# import mrcnn.model     as modellib
import mrcnn.visualize as visualize
# import mrcnn.shapes    as shapes
import mrcnn.shapes as shapes
from mrcnn.config      import Config
from mrcnn.model       import log
from mrcnn.dataset     import Dataset 

from mrcnn.utils       import stack_tensors, stack_tensors_3d
from mrcnn.datagen     import data_generator, load_image_gt
from mrcnn.callbacks   import get_layer_output_1,get_layer_output_2
# from mrcnn.visualize   import plot_gaussian
# from mrcnn.pc_layer    import PCTensor
# from mrcnn.pc_layer   import PCNLayer

# Root directory of the project
ROOT_DIR = os.getcwd()
MODEL_PATH = 'E:\Models'
# Directory to save logs and trained model
MODEL_DIR = os.path.join(MODEL_PATH, "mrcnn_logs")
# Path to COCO trained weights
COCO_MODEL_PATH   = os.path.join(MODEL_PATH, "mask_rcnn_coco.h5")
RESNET_MODEL_PATH = os.path.join(MODEL_PATH, "resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")

# print("Tensorflow Version: {}   Keras Version : {} ".format(tf.__version__,keras.__version__))
pp = pprint.PrettyPrinter(indent=2, width=100)
np.set_printoptions(linewidth=100,precision=4)


# Build configuration object -----------------------------------------------
config = shapes.ShapesConfig()
config.BATCH_SIZE      = 5                  # Batch size is 2 (# GPUs * images/GPU).
config.IMAGES_PER_GPU  = 5                  # Must match BATCH_SIZE
config.STEPS_PER_EPOCH = 2
config.FCN_INPUT_SHAPE = config.IMAGE_SHAPE[0:2]

# config.LAST_EPOCH_RAN  = 5686  # <---- if we want to continue training from a previously run
config.display() 

Using TensorFlow backend.


ImportError: cannot import name 'CHMLayerInference'

In [ ]:
# Build shape dataset        -----------------------------------------------
# Training dataset
# generate 500 shapes 
dataset_train = shapes.ShapesDataset()
dataset_train.load_shapes(10, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
dataset_train.prepare()

# Validation dataset
# dataset_val = shapes.NewShapesDataset()
# dataset_val.load_shapes(50, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
# dataset_val.prepare()

In [ ]:
# Load and display random samples
# image_ids = np.random.choice(dataset_train.image_ids, 3)
# for image_id in [3]:
#     image = dataset_train.load_image(image_id)
#     mask, class_ids = dataset_train.load_mask(image_id)
#     visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

In [ ]:
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, 4)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names, limit=6)

In [ ]:
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, 4)
for image_id in [396]:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names, limit=6)

###  Define data generators 

In [ ]:
train_generator = data_generator(dataset_train, config, shuffle=True, batch_size=6, augment = False)

# val_generator = data_generator(dataset_val, model.config, shuffle=True, batch_size=model.config.BATCH_SIZE,  augment=False)

In [ ]:
train_batch_x, train_batch_y = next(train_generator)

In [ ]:
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, 4)
for image_id in [396]:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)c
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names, limit=6)

In [ ]:
np.set_printoptions(linewidth=135,precision=4,threshold=20000)
for image_id in [8]:
#     print('Classes (1: circle, 2: square, 3: triangle ): ',class_ids)
    image, image_meta, gt_class_ids, gt_boxes, gt_masks = \
                    load_image_gt(dataset_train, config, image_id, augment=False, use_mini_mask=False) 
    print(gt_class_ids.shape, gt_boxes.shape, gt_masks.shape)
    for i in range(gt_masks.shape[-1]):
        print()
        print(np.array2string(np.where(gt_masks[:,:,i],1,0),max_line_width=134, separator = ''))
    print(gt_class_ids)        
    visualize.display_images([image], cols = 1, width = 8)     

####  Image info structure 

In [ ]:
dataset_train.image_info[8]

In [ ]:

img_id = 8
shapes = []
boxes = []
for tmp_shapes in dataset_train.image_info[img_id]['shapes']:
    shapes.append(tmp_shapes)
    x, y, sx, sy = tmp_shapes[2]
    boxes.append([y - sy, x - sx, y + sy, x + sx])            
print(shapes)
boxes_np = np.array(boxes)
print(boxes_np)

In [ ]:
from mrcnn.new_shapes import debug_non_max_suppression
ixs = debug_non_max_suppression(boxes_np, np.arange(boxes_np.shape[0]), 0.29)

### Define Data Generator

In [ ]:
train_generator = data_generator(dataset_train, model.config, shuffle=True,
                                 batch_size=model.config.BATCH_SIZE,
                                 augment = False)
val_generator = data_generator(dataset_val, model.config, shuffle=True, 
                                batch_size=model.config.BATCH_SIZE,
                                augment=False)

### Get next shapes from generator and display loaded shapes

In [ ]:
train_batch_x, train_batch_y = next(train_generator)

In [ ]:
# train_batch_x, train_batch_y = next(train_generator)
imgmeta_idx = model.keras_model.input_names.index('input_image_meta')
img_meta    = train_batch_x[imgmeta_idx]

for img_idx in range(config.BATCH_SIZE):
    image_id = img_meta[img_idx,0]
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    print('Image id: ',image_id)
    print('Image meta', img_meta[img_idx])
    print('Classes (1: circle, 2: square, 3: triangle ): ',class_ids)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)



### Display ground truth bboxes from Shapes database (using `load_image_gt` )

Here we are displaying the ground truth bounding boxes as provided by the dataset

In [ ]:
img = 0
image_id = img_meta[img,0]
print('Image id: ',image_id)
p_original_image, p_image_meta, p_gt_class_id, p_gt_bbox, p_gt_mask =  \
            load_image_gt(dataset_train, config, image_id, augment=False, use_mini_mask=True)
# print(p_gt_class_id.shape, p_gt_bbox.shape, p_gt_mask.shape)
print(p_gt_bbox[0:3,:])
print(p_gt_class_id)
visualize.draw_boxes(p_original_image, p_gt_bbox[0:3])

# image_id = img_meta[img,0]
# print('Image id: ',image_id)
# p_original_image, p_image_meta, p_gt_class_id, p_gt_bbox, p_gt_mask =  \
#             load_image_gt(dataset_train, config, image_id, augment=False, use_mini_mask=True)
# # print(p_gt_class_id.shape, p_gt_bbox.shape, p_gt_mask.shape)
# print(p_gt_bbox)
# print(p_gt_class_id)
# visualize.draw_boxes(p_original_image, p_gt_bbox)